<a href="https://colab.research.google.com/github/kmk4444/Langchain/blob/main/Loaders_Splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Langchain provides more than one loaders but we focus on three loaders.
1. WebBaseLoader - Load from a web page
2. PyPDFLoader - Load data from a pdf
3. UnstructuredExcelLoader - Load data from a excel

Document loader means that several structure data converts into documents. (Document.page_content and Document.metada)

**Requirements.txt**

In [4]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchain_experimental >> requirements.txt
!echo openai >> requirements.txt
!echo anthropic >> requirements.txt
!echo cohere >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo pypdf >> requirements.txt
!echo unstructured >> requirements.txt
!echo networkx >> requirements.txt
!echo openpyxl >> requirements.txt
!echo rapidocr-onnxruntime >> requirements.txt

**terminal / bash komutu**

In [ ]:
pip install -r requirements.txt

In [9]:
# WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader

target_url="https://kpmg.com/tr/tr/home/gorusler/2023/12/uretken-yapay-zeka-uygulamalarinin-kurumsallasma-yaklasimi.html"

loader = WebBaseLoader(target_url) # we create a loader object and we assign to this object.
raw_documents = loader.load() # we load document from url. (type is document)


with open("URL_Icerik.txt","w") as file: # we create a file to save raw_documents for string format.
  file.write(raw_documents[0].page_content) #page_content is information we want.

print("Dosya işlemi tamamlandı.")

print(raw_documents[0].metadata) # we can design metadata in RAG.

# PyPDFLoader - Bir PDF dosyasından içerik yükleme
from langchain_community.document_loaders import PyPDFLoader

#first file
filepath = "/content/drive/MyDrive/timeline.pdf"
loader = PyPDFLoader(filepath) # we are ready to load our file
pages = loader.load() # we load our pdf

print(pages[39].page_content,pages[39].metadata) # index 39 means that 40th page

#second file
filepath =


Dosya işlemi tamamlandı.
{'source': 'https://kpmg.com/tr/tr/home/gorusler/2023/12/uretken-yapay-zeka-uygulamalarinin-kurumsallasma-yaklasimi.html', 'title': 'Üretken Yapay Zeka Uygulamalarının Kurumsallaşma Yaklaş - KPMG Türkiye', 'description': 'Mikro-Dil-Modeli Mimarisi', 'language': 'tr-TR'}
 
 
1989: George Cybenko proves that neural networks can approximate continuous 
functions  
 
 
1989: Yann LeCun's convolutional neural network for handwritten -digit recognition 
(LeNet -1)  
 
 
 
1989: Kurt Hornik proves that neural networks are universal app roximators  
 
 
 
1990: Carver Mead describes a neuromorph ic processor   {'source': '/content/drive/MyDrive/timeline.pdf', 'page': 39}
